In [1]:
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import StructField, StructType, StringType, LongType
from pyspark.sql.functions import *
from pyspark.sql import Row

In [2]:
spark = SparkSession.builder.appName("spark_definitive_guide").getOrCreate()

22/10/09 14:35:45 WARN Utils: Your hostname, AS-MAC-0006.local resolves to a loopback address: 127.0.0.1; using fd01:db8:1111:0:0:0:0:3 instead (on interface lo0)
22/10/09 14:35:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/09 14:35:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
path = "./../src/retail-data/by-day/2010-12-01.csv"
df = spark.read.format('csv').option("header", "true").option("inferSchema", "true").load(path)
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [17]:
df.where("InvoiceNo <> 536365").selectExpr("InvoiceNo", "Description").show()

+---------+--------------------+
|InvoiceNo|         Description|
+---------+--------------------+
|   536366|HAND WARMER UNION...|
|   536366|HAND WARMER RED P...|
|   536367|ASSORTED COLOUR B...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|POPPY'S PLAYHOUSE...|
|   536367|FELTCRAFT PRINCES...|
|   536367|IVORY KNITTED MUG...|
|   536367|BOX OF 6 ASSORTED...|
|   536367|BOX OF VINTAGE JI...|
|   536367|BOX OF VINTAGE AL...|
|   536367|HOME BUILDING BLO...|
|   536367|LOVE BUILDING BLO...|
|   536367|RECIPE BOX WITH M...|
|   536367| DOORMAT NEW ENGLAND|
|   536368|JAM MAKING SET WI...|
|   536368|RED COAT RACK PAR...|
|   536368|YELLOW COAT RACK ...|
|   536368|BLUE COAT RACK PA...|
|   536369|BATH BUILDING BLO...|
|   536370|ALARM CLOCK BAKEL...|
+---------+--------------------+
only showing top 20 rows



22/10/09 18:23:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1933814 ms exceeds timeout 120000 ms
22/10/09 18:23:35 WARN SparkContext: Killing executors is not supported by current scheduler.


In [25]:
# applying multiple filters into one liner

DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1 # like string contains given subString 1 or more times
newQuality = pow(col("Quantity") * col("UnitPrice"), 2) + 5 # like an expression

df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter)).where("isExpensive")\
  .withColumn("realQuantity", newQuality)\
  .show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+------------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|      realQuantity|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+------------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|       true|       324642.8529|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|       true|369049.10010000004|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+------------------+



In [28]:
# print(df.stat.corr("Quantity", "UnitPrice"))
df.select(corr("Quantity", "UnitPrice")).show()
df.describe().show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [34]:
df.select(monotonically_increasing_id()).show()

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
|                            9|
|                           10|
|                           11|
|                           12|
|                           13|
|                           14|
|                           15|
|                           16|
|                           17|
|                           18|
|                           19|
+-----------------------------+
only showing top 20 rows

